ergrant jborromeo SYDE 552 Final Project

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import keras_spiking

tf.random.set_seed(0)
np.random.seed(0)

In [4]:
# Load CIFAR-10 Dataset (Colour Images)
((train_images, train_labels),(test_images, test_labels),) = tf.keras.datasets.cifar10.load_data()
train_labels = train_labels.squeeze()
test_labels = test_labels.squeeze()

# maximum of each colour is 255
#print(np.max(train_images[0], axis=(0,1)))

# normalize images so values are between 0 and 1
train_images = train_images / 255.0
test_images = test_images / 255.0

class_names = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]
num_classes = len(class_names)

## Non-Spiking Model for Comparison

In [5]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(32, 32,3)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(10),
    ]
)


def train(input_model, train_x, test_x):
    input_model.compile(
        optimizer="adam",
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )

    input_model.fit(train_x, train_labels, epochs=10)

    _, test_acc = input_model.evaluate(test_x, test_labels, verbose=2)

    print("\nTest accuracy:", test_acc)


train(model, train_images, test_images)

Epoch 1/10
1563/1563 [==============================] - 2s 1ms/step - loss: 2.0217 - accuracy: 0.2724
Epoch 2/10
1563/1563 [==============================] - 2s 1ms/step - loss: 1.8070 - accuracy: 0.3539
Epoch 3/10
1563/1563 [==============================] - 2s 1ms/step - loss: 1.7416 - accuracy: 0.3782
Epoch 4/10
1563/1563 [==============================] - 2s 1ms/step - loss: 1.7116 - accuracy: 0.3905
Epoch 5/10
1563/1563 [==============================] - 2s 1ms/step - loss: 1.6810 - accuracy: 0.4005
Epoch 6/10
1563/1563 [==============================] - 2s 1ms/step - loss: 1.6805 - accuracy: 0.4000
Epoch 7/10
1563/1563 [==============================] - 2s 1ms/step - loss: 1.6633 - accuracy: 0.4056
Epoch 8/10
1563/1563 [==============================] - 2s 1ms/step - loss: 1.6590 - accuracy: 0.4065
Epoch 9/10
1563/1563 [==============================] - 2s 1ms/step - loss: 1.6463 - accuracy: 0.4104
Epoch 10/10
1563/1563 [==============================] - 2s 1ms/step - loss: 1.646

## Conversion to Spiking Model

In [6]:
# repeat the images for n_steps
n_steps = 10
train_sequences = np.tile(train_images[:, None], (1, n_steps, 1, 1, 1))
test_sequences = np.tile(test_images[:, None], (1, n_steps, 1, 1, 1))

In [7]:
spiking_model = tf.keras.Sequential(
    [
        # add temporal dimension to the input shape; we can set it to None,
        # to allow the model to flexibly run for different lengths of time
        tf.keras.layers.Reshape((-1, 32 * 32 * 3), input_shape=(None, 32, 32, 3)),
        # we can use Keras' TimeDistributed wrapper to allow the Dense layer
        # to operate on temporal data
        tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(128)),
        # replace the "relu" activation in the non-spiking model with a
        # spiking equivalent
        keras_spiking.SpikingActivation("relu", spiking_aware_training=False),
        # use average pooling layer to average spiking output over time
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(10),
    ]
)

# train the model, identically to the non-spiking version,
# except using the time sequences as inputs
train(spiking_model, train_sequences, test_sequences)

KeyboardInterrupt: 